# TFM - Big Data Analytics

## Generación datos sintéticos - Experimento 2 (Turbidez)

### Virginia Casino Sánchez (vcassan@disca.upv.es)

## Librerías

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from scipy.spatial.distance import jensenshannon
from scipy.stats import wasserstein_distance
from scipy.stats import ks_2samp

## Parámetros a configurar

In [ ]:
exp = 2
prueba = 7

scaler_on = False
scaler_type = MaxAbsScaler()
# MaxAbsScaler()
# StandardScaler()
# MinMaxScaler()
# RobustScaler()

noise_dim = 6
feature_dim = 1
time_steps = 6

epochs= 500
batch_size= 12

modelo = 'LSTM_Dropout'
# 'Basic'
# 'Complex'
# 'LeakyRELU'
# 'Dropout'
# 'LSTM'
# 'GRU'
# 'LeakyRELU_LSTM'
# 'LSTM_Dropout'

leaky_relu_alpha = 0.2
leaky_relu_negative_slope = 0.2
dropout= 0.3

lr = 0.0001
opt = tf.keras.optimizers.RMSprop(learning_rate=lr)
# tf.keras.optimizers.Adam(learning_rate=lr)
# tf.keras.optimizers.Adagrad(learning_rate=lr)
# tf.keras.optimizers.SGD(learning_rate=lr)
# tf.keras.optimizers.RMSprop(learning_rate=0.00001) #0.00005

loss = 'binary_crossentropy'
# 'binary_crossentropy'
# 'wasserstein'
# 'weighted_loss'

#weighted_loss
threshold = 10.0
factor = 2.0

# Establecer la semilla para reproducibilidad
np.random.seed(20)

In [ ]:
if loss != 'weighted_loss':
    threshold = np.NaN
    factor = np.NaN

if not scaler_on:
    scaler_type = None

## Carga y preparación de los datos

In [ ]:
# Cargar los datos
data = pd.read_csv('data_turb.csv')

# Asegurar que la columna 'date' está en formato datetime y generar las fechas faltantes
data['date'] = pd.to_datetime(data['date'])

# Crear un rango de fechas diarias desde la primera hasta la última fecha del dataset
full_date_range = pd.date_range(start=data['date'].min(), end=data['date'].max(), freq='D')

# Reindexar los datos para incluir todas las fechas en el rango, asignando NaN a las fechas faltantes
data_full = data.set_index('date').reindex(full_date_range).reset_index()
data_full.columns = ['date', 'turb']

if scaler_on:
  # Escalar los valores de 'turb' usando la normalización especificada
  scaler = scaler_type
  # Escalar solo los valores no nulos
  data_full['turb_scaled'] = scaler.fit_transform(data_full[['turb']])
else:
  # No escalar los valores
  data_full['turb_scaled'] = data_full[['turb']]

# Reemplazar los NaN por 0 en los datos escalados, pero mantenerlos marcados para luego rellenarlos
nan_mask = data_full['turb_scaled'].isna()
data_full['turb_scaled'].fillna(0, inplace=True)

## Definir arquitectura y entrenamiento de la GAN

In [ ]:
# Definir el generador
def build_generator(modelo,
                    noise_dim,
                    feature_dim,
                    time_steps,
                    leaky_relu_alpha,
                    leaky_relu_negative_slope,
                    dropout):

    if modelo == 'Basic':
      # Modelo básico
      model = tf.keras.Sequential()

      # Primera capa densa
      model.add(layers.Dense(128, activation='relu', input_dim=noise_dim))

      # Segunda capa
      model.add(layers.Dense(256, activation='relu'))

      # Capa de salida
      model.add(layers.Dense(feature_dim, activation='tanh'))

    elif modelo == 'Complex':
      # Modelo complejo
      model = tf.keras.Sequential()

      # Primera capa
      model.add(layers.Dense(256, activation='relu', input_dim=noise_dim))

      # Segunda capa con más neuronas
      model.add(layers.Dense(512, activation='relu'))

      # Añadir una tercera capa
      model.add(layers.Dense(1024, activation='relu'))

      # Capa de salida
      model.add(layers.Dense(feature_dim, activation='tanh'))

    elif modelo == 'LeakyRELU':
      model = tf.keras.Sequential()
      model.add(layers.Dense(128, input_dim=noise_dim))
      model.add(layers.LeakyReLU(alpha=leaky_relu_alpha))  # Leaky ReLU permite pequeñas pendientes negativas

      # Aumentar la complejidad del modelo
      model.add(layers.Dense(256))
      model.add(layers.LeakyReLU(alpha=leaky_relu_alpha))

      # Más capas densas
      model.add(layers.Dense(512))
      model.add(layers.LeakyReLU(alpha=leaky_relu_alpha))

      # Capa de salida
      model.add(layers.Dense(feature_dim, activation='tanh'))

    elif modelo == 'Dropout':
      model = tf.keras.Sequential()

      # Primera capa con Dropout
      model.add(layers.Dense(256, activation='relu', input_dim=noise_dim))
      model.add(layers.Dropout(dropout))  # Dropout con una tasa del 30%

      # Segunda capa con más neuronas y Dropout
      model.add(layers.Dense(512, activation='relu'))
      model.add(layers.Dropout(dropout))  # Otro Dropout

      # Tercera capa
      model.add(layers.Dense(1024, activation='relu'))

      # Capa de salida
      model.add(layers.Dense(feature_dim, activation='tanh'))  # Output con activación 'tanh'

    elif modelo == 'LSTM':
      model = tf.keras.Sequential()

      # Expandir la dimensión del ruido para que coincida con (batch_size, time_steps, feature_dim)
      model.add(layers.Dense(time_steps * feature_dim, input_dim=noise_dim))
      model.add(layers.Reshape((time_steps, feature_dim)))

      # Añadir la capa LSTM
      model.add(layers.LSTM(256, return_sequences=False))
      model.add(layers.Dense(512, activation='relu'))

      # Capa de salida
      model.add(layers.Dense(feature_dim, activation='tanh'))

    elif modelo == 'GRU':
      model = tf.keras.Sequential()
      # Expandir la dimensión del ruido para que coincida con (batch_size, time_steps, feature_dim)
      model.add(layers.Dense(time_steps * feature_dim, input_dim=noise_dim))
      model.add(layers.Reshape((time_steps, feature_dim)))

      # Cambiar la capa Dense a una capa GRU
      model.add(layers.GRU(256, return_sequences=False, input_shape=(time_steps, feature_dim)))
      model.add(layers.Dense(512, activation='relu'))

      # Capa de salida
      model.add(layers.Dense(feature_dim, activation='tanh'))

    elif modelo == 'LSTM_Dropout':
      model = tf.keras.Sequential()

      # Capa densa inicial que convierte el ruido en una forma adecuada
      model.add(layers.Dense(time_steps * feature_dim, input_dim=noise_dim))
      model.add(layers.Reshape((time_steps, feature_dim)))  # Reformatear a 3D para LSTM

      # Capas LSTM con Dropout
      model.add(layers.LSTM(256, return_sequences=True))
      model.add(layers.Dropout(dropout))  # Aplicar Dropout del 30%

      model.add(layers.LSTM(128, return_sequences=False))
      model.add(layers.Dropout(dropout))  # Otro Dropout

      # Capa densa para generar salida
      model.add(layers.Dense(512, activation='relu'))

      # Capa de salida
      model.add(layers.Dense(feature_dim, activation='tanh'))

    elif modelo == 'LeakyRELU_LSTM':
      model = tf.keras.Sequential()

      # Capa inicial para expandir el ruido a la forma (time_steps, feature_dim)
      model.add(layers.Dense(time_steps * feature_dim, input_dim=noise_dim))
      model.add(layers.Reshape((time_steps, feature_dim)))

      # Capa LSTM
      model.add(layers.LSTM(256, return_sequences=True))
      model.add(layers.LeakyReLU(negative_slope=leaky_relu_negative_slope))

      # Segunda capa LSTM o Dense
      model.add(layers.LSTM(128, return_sequences=False))
      model.add(layers.LeakyReLU(negative_slope=leaky_relu_negative_slope))

      # Capa de salida para generar secuencias
      model.add(layers.Dense(feature_dim, activation='tanh'))
    return model

In [ ]:
# Definir el discriminador
def build_discriminator(modelo,
                        feature_dim,
                        time_steps,
                        leaky_relu_alpha,
                        leaky_relu_negative_slope,
                        dropout):
    if modelo == 'Basic':
      model = tf.keras.Sequential()
      model.add(layers.Dense(128, activation='relu', input_dim=feature_dim))  # El discriminador recibe un valor continuo
      model.add(layers.Dense(256, activation='relu'))
      model.add(layers.Dense(1, activation='sigmoid'))  # Salida binaria: real o falso

    elif modelo == 'Complex':
      model = tf.keras.Sequential()

      # Aumentar la complejidad con capas adicionales
      model.add(layers.Dense(512, input_dim=feature_dim, activation='relu'))
      model.add(layers.Dense(256, activation='relu'))
      model.add(layers.Dense(128, activation='relu'))

      # Capa de salida para clasificación
      model.add(layers.Dense(1, activation='sigmoid'))  # Salida binaria: real o falso
    elif modelo == 'LeakyRELU':
      model = tf.keras.Sequential()

      # Usar LeakyReLU en lugar de ReLU
      model.add(layers.Dense(512, input_dim=feature_dim))
      model.add(layers.LeakyReLU(alpha=leaky_relu_alpha))

      model.add(layers.Dense(256))
      model.add(layers.LeakyReLU(alpha=leaky_relu_alpha))

      model.add(layers.Dense(128))
      model.add(layers.LeakyReLU(alpha=leaky_relu_alpha))

      # Capa de salida para clasificación
      model.add(layers.Dense(1, activation='sigmoid'))  # Salida binaria: real o falso
    elif modelo == 'Dropout':
      model = tf.keras.Sequential()

      # Primera capa con Dropout
      model.add(layers.Dense(512, input_dim=feature_dim, activation='relu'))
      model.add(layers.Dropout(dropout))  # Dropout con una tasa del 30%

      # Segunda capa con más neuronas y Dropout
      model.add(layers.Dense(256, activation='relu'))
      model.add(layers.Dropout(dropout))

      # Tercera capa
      model.add(layers.Dense(128, activation='relu'))

      # Capa de salida para clasificación
      model.add(layers.Dense(1, activation='sigmoid'))  # Salida binaria: real o falso
    elif modelo == 'LSTM':
      model = tf.keras.Sequential()

      # Capa LSTM
      model.add(layers.LSTM(256, return_sequences=False, input_shape=(time_steps, feature_dim)))

      # Capa densa
      model.add(layers.Dense(128, activation='relu'))

      # Capa de salida para clasificación
      model.add(layers.Dense(1, activation='sigmoid'))  # Salida binaria: real o falso
    elif modelo == 'GRU':
      model = tf.keras.Sequential()

      # Capa GRU
      model.add(layers.GRU(256, return_sequences=False, input_shape=(time_steps, feature_dim)))

      # Capa densa
      model.add(layers.Dense(128, activation='relu'))

      # Capa de salida para clasificación
      model.add(layers.Dense(1, activation='sigmoid'))  # Salida binaria: real o falso
    elif modelo == 'LSTM_Dropout':
      model = tf.keras.Sequential()

      # Capa LSTM con Dropout
      model.add(layers.Input(shape=(time_steps, feature_dim)))  # Definir la forma de la entrada
      model.add(layers.LSTM(256, return_sequences=True))
      model.add(layers.Dropout(dropout))  # Aplicar Dropout del 30%

      # Otra capa LSTM
      model.add(layers.LSTM(128, return_sequences=False))
      model.add(layers.Dropout(dropout))  # Otro Dropout

      # Capa densa
      model.add(layers.Dense(128, activation='relu'))

      # Capa de salida binaria
      model.add(layers.Dense(1, activation='sigmoid'))  # Clasificación binaria (real o falso)
    elif modelo == 'LeakyRELU_LSTM':
      model = tf.keras.Sequential()

      # Capa LSTM
      model.add(layers.LSTM(256, return_sequences=True, input_shape=(time_steps, feature_dim)))
      model.add(layers.LeakyReLU(negative_slope=leaky_relu_negative_slope))

      # Capa adicional de LSTM o Dense (si deseas más capas antes de la salida final)
      model.add(layers.LSTM(128, return_sequences=False))
      model.add(layers.LeakyReLU(negative_slope=leaky_relu_negative_slope))

      # Capa de salida para la clasificación binaria (real o falso)
      model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [ ]:
# Definir el entrenamiento
def train_gan(generator, discriminator, gan, data, mean, std, epochs, batch_size, lstm_mode):
    for epoch in range(epochs):
        if lstm_mode:
          # Seleccionar un batch de datos reales (con las dimensiones correctas)
          idx = np.random.randint(0, data.shape[0], batch_size)
          real_data = data[idx]  # Seleccionar las secuencias reales (batch de secuencias)

          # Generar ruido para el generador
          noise = np.random.normal(mean, std, (batch_size, generator.input_shape[-1]))

        else:
          # Seleccionar valores reales aleatorios de los datos
          real_data = np.random.choice(data, batch_size)
          # Generar ruido y valores falsos
          noise = np.random.normal(mean, std, (batch_size, noise_dim))

        # Generar datos falsos (generados) a partir del ruido
        generated_data = generator.predict(noise)

        # Entrenar el discriminador en datos reales y falsos
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        # Entrenar discriminador en datos reales y falsos
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)  # 1 para reales
        d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)  # 0 para falsos

        # Entrenamiento del generador a través del GAN completo
        g_loss = gan.train_on_batch(noise, real_labels)

        # Monitoreo de progreso en intervalos de epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs} - D Loss Real: {d_loss_real} - D Loss Fake: {d_loss_fake} - G Loss: {g_loss}")

In [ ]:
# Generar datos sintéticos
def fill_missing_values(generator, nan_mask, mean, std):
    # Crear ruido aleatorio para rellenar los valores faltantes
    missing_count = nan_mask.sum()

    noise = np.random.normal(mean, std, (missing_count, noise_dim))

    # Usar el generador para predecir los valores para las fechas faltantes
    generated_values = generator.predict(noise)

    return generated_values[:, 0]

In [ ]:
# Definir la pérdida de Wasserstein
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

def weighted_loss(y_true, y_pred):
    # Convertimos la condición booleana a float32
    weight = 1 + K.cast(y_true > threshold, K.floatx()) * factor

    # Calculamos la pérdida ponderada
    return K.mean(weight * K.square(y_true - y_pred))

In [ ]:
# Crear las secuencias de datos
def create_sequences(data, time_steps):
    sequences = []
    for i in range(len(data) - time_steps):
        # Cada secuencia tendrá time_steps de largo
        seq = data[i:i + time_steps]
        sequences.append(seq)
    return np.array(sequences)

In [ ]:
# Crear los modelos
generator = build_generator(modelo, noise_dim, feature_dim, time_steps, leaky_relu_alpha,leaky_relu_negative_slope, dropout)
discriminator = build_discriminator(modelo, feature_dim, time_steps, leaky_relu_alpha,leaky_relu_negative_slope, dropout)

# Compilar los modelos
if loss == 'wasserstein_loss':
  discriminator.compile(optimizer=opt, loss=wasserstein_loss, metrics=['accuracy'])
elif loss == 'weighted_loss':
  discriminator.compile(optimizer=opt, loss=weighted_loss, metrics=['accuracy'])
else:
  discriminator.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

# El GAN es una combinación de los dos modelos, donde solo se entrena el generador
discriminator.trainable = False

# Entrada para el GAN: ruido aleatorio
gan_input = layers.Input(shape=(noise_dim,))
generated_value = generator(gan_input)
gan_output = discriminator(generated_value)

gan = tf.keras.Model(gan_input, gan_output)

if loss == 'wasserstein_loss':
  gan.compile(optimizer=opt, loss=wasserstein_loss)
elif loss == 'weighted_loss':
  gan.compile(optimizer=opt, loss=weighted_loss)
else:
  gan.compile(optimizer=opt, loss=loss)

In [ ]:
# Generar el diagrama del generador
plot_model(generator, to_file=f'generator_exp{exp}_prueba{prueba}.png', show_shapes=True, show_layer_names=False)

# Generar el diagrama del discriminador
plot_model(discriminator, to_file=f'discriminator_exp{exp}_prueba{prueba}.png', show_shapes=True, show_layer_names=False)

# Generar el diagrama del generador
plot_model(gan, to_file=f'gan_exp{exp}_prueba{prueba}.png', show_shapes=True, show_layer_names=False)


In [ ]:
# Entrenar el GAN
turb_mean = data_full['turb'][~nan_mask].mean()
turb_std = data_full['turb'][~nan_mask].std()

if modelo == 'LSTM' or modelo == 'GRU' or modelo == 'LSTM_Dropout' or modelo == 'LeakyRELU_LSTM':
  lstm_mode = True
  # Usar esta función para transformar los datos de entrenamiento
  data_sequences = create_sequences(data_full['turb_scaled'][~nan_mask].values, time_steps)

  # Asegúrate de que los datos tengan la forma correcta: (n_samples, time_steps, feature_dim)
  data_sequences = data_sequences.reshape(-1, time_steps, feature_dim)

  # Ejecutar el entrenamiento (usamos los valores escalados que no son NaN)
  train_gan(generator, discriminator, gan, data_sequences, turb_mean, turb_std, epochs, batch_size, lstm_mode)
else:
  lstm_mode = False
  # Ejecutar el entrenamiento (usamos los valores escalados que no son NaN)
  train_gan(generator, discriminator, gan, data_full['turb_scaled'][~nan_mask].values, turb_mean, turb_std, epochs, batch_size, lstm_mode)

# Utilizar el generador entrenado para predecir los valores faltantes
generated_values_scaled = fill_missing_values(generator, nan_mask, turb_mean, turb_std)

# Reemplazar los valores faltantes escalados por los generados
data_full.loc[nan_mask, 'turb_scaled'] = generated_values_scaled

if scaler_on:
  # Reescalar los valores generados a su rango original
  data_full['turb_filled'] = scaler.inverse_transform(data_full[['turb_scaled']])
else:
  data_full['turb_filled'] = data_full[['turb_scaled']]

data_full['turb_new'] = data_full['turb_filled']
data_full.loc[data_full['turb'].notna(), 'turb_new'] = np.nan

## Visualizar resultados

In [ ]:
def pintar_grafica(titulo, file_svg, file_png, type):
    ancho = 15
    alto = 10
    label_x = 'Fecha'
    label_y = 'Turbidez'

    # Preparación gráfica
    fig, ax = plt.subplots(figsize=(ancho, alto))

    if type == 'scatter':
        # Crear el gráfico de scatter
        plt.scatter(data_full['date'], data_full['turb'], label='Entrenamiento', color='royalblue', marker='o')
        plt.scatter(data_full['date'], data_full['turb_new'], label='Generados (reconstrucción)', color='seagreen', marker='x')
    elif type == 'unificado':
      # Crear el gráfico todos los valores unificados
        plt.plot(data_full['date'], data_full['turb_filled'], color='royalblue')
    elif type == 'histograma':
      # Crear el histograma
      plt.hist([data_full['turb'], data_full['turb_new']],
               label=["Real", "Sintético"],
               bins=25,
               density=True, color=['royalblue','seagreen'])
    else:
        # Crear el gráfico de líneas
        plt.plot(data_full['date'], data_full['turb'], label='Entrenamiento', color='royalblue')
        plt.plot(data_full['date'], data_full['turb_new'], label='Generados (reconstrucción)', linestyle='--', color='seagreen')

    # Configuración a los ejes / grid
        # Color de fondo
    ax.set_facecolor('white')

        # Configuración del grid
    ax.grid(True, color='lightgrey', linestyle='--', linewidth=0.7, axis='y', which='major')
    ax.yaxis.grid(True, color='lightgrey', linestyle='--', linewidth=0.2, which='minor')

        # Configuración de los ticks del eje y
    ax.yaxis.set_major_locator(ticker.MultipleLocator(10))   # Ticks mayores cada 1
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(50)) # Ticks menores cada 0.5

    ax.tick_params(axis='y', labelsize=25)

        # Configuración de los ticks del eje x
    ax.tick_params(axis='x', which='major', labelsize=25)

    # Leyenda
    ax.legend(fontsize=20)

    # Configuración de las etiquetas
    ax.set_xlabel(label_x, fontsize=30, labelpad=25)
    ax.set_ylabel(label_y, fontsize=30, labelpad=25)
    ax.set_title(titulo, fontsize=35, pad=10)

    # Borrar líneas superior y derecha
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Ajustar imagen
    plt.tight_layout()

    # Guardar imagen
    plt.savefig(file_svg, format='svg')
    plt.savefig(file_png, format='png')

    # Mostrar imagen
    plt.show()

In [ ]:
titulo = f'Scatter Experimento {exp} Prueba {prueba}'
file_svg_scatter = f'scatter_{exp}_{prueba}.svg'
file_png_scatter = f'scater_{exp}_{prueba}.png'
type = 'scatter'

pintar_grafica(titulo, file_svg_scatter, file_png_scatter, type)

In [ ]:

titulo = f'Resultados Experimento {exp} Prueba {prueba}'
file_svg_res = f'resultado_{exp}_{prueba}.svg'
file_png_res = f'resultado_{exp}_{prueba}.png'

type = 'line'

pintar_grafica(titulo, file_svg_res, file_png_res, type)

In [ ]:
titulo = f'Unificado Experimento {exp} Prueba {prueba}'
file_svg_union = f'union_{exp}_{prueba}.svg'
file_png_union = f'union_{exp}_{prueba}.png'

type = 'unificado'

pintar_grafica(titulo, file_svg_union, file_png_union, type)

In [ ]:
titulo = f'Histograma Experimento {exp} Prueba {prueba}'
file_svg_histo = f'histo_{exp}_{prueba}.svg'
file_png_histo = f'histo_{exp}_{prueba}.png'

type = 'histograma'

pintar_grafica(titulo, file_svg_histo, file_png_histo, type)

## Obtener resultados numéricos

In [ ]:
# Extraer los valores reales y generados del dataset
real_values = data_full['turb'].dropna().values
generated_values = data_full['turb_filled'].dropna().values


# Funciones de evaluación para comparar datos reales y generados

# Distancia de Jensen-Shannon: mide la diferencia entre distribuciones
def calculate_jsd(real_data, generated_data, bins=50):
    # Obtener el rango común para ambos conjuntos de datos
    data_min = min(real_data.min(), generated_data.min())
    data_max = max(real_data.max(), generated_data.max())

    # Crear bin edges basados en este rango común
    bins = np.linspace(data_min, data_max, 100)  # Número de bins configurable

    # Generar histogramas usando los mismos bin edges
    # Histograma de los datos reales y generados
    real_hist, bin_edges = np.histogram(real_data, bins=bins, density=True)
    generated_hist, _ = np.histogram(generated_data, bins=bins, density=True)

    # Normalizar las distribuciones para que sumen 1
    real_hist = real_hist / np.sum(real_hist)
    generated_hist = generated_hist / np.sum(generated_hist)

    # Calcular la distancia de Jensen-Shannon
    jsd = jensenshannon(real_hist, generated_hist, base=2)
    return jsd

def calculate_wd(real_data, generated_data):
  return wasserstein_distance(real_data, generated_data)

def calculate_ks(real_data, generated_data):
  return ks_2samp(real_data, generated_data)

# Calcular la distancia de Jensen-Shannon
jsd_value = calculate_jsd(real_values, generated_values)
print(f"Distancia de Jensen-Shannon: {jsd_value}")
print()

# Calcular la distancia de Wasserstein
wasserstein_dist = calculate_wd(real_values, generated_values)
print(f"Wasserstein Distance: {wasserstein_dist}")
print()

# Realizar el Kolmogorov-Smirnov Test
ks_stat, p_value = calculate_ks(real_values, generated_values)
print(f"K-S Statistic: {ks_stat}")
print(f"P-Value: {p_value}")
print()


## Guardar resultados

In [ ]:
file_models = f'model_summaries_{exp}_{prueba}.txt'

with open(file_models, 'w', encoding='utf-8') as f:
    # Guardar resumen del generador
    f.write("Resumen del Generador:\n")
    generator.summary(print_fn=lambda x: f.write(x + '\n'))

    # Guardar resumen del discriminador
    f.write("\nResumen del Discriminador:\n")
    discriminator.summary(print_fn=lambda x: f.write(x + '\n'))

    # Guardar resumen de la GAN
    f.write("\nResumen de la GAN:\n")
    gan.summary(print_fn=lambda x: f.write(x + '\n'))

In [ ]:
conf_res_data = {
    'experimento': [exp],
    'prueba': [prueba],
    'normalizado': [scaler_on],
    'tipo_normalizacion': [scaler_type],
    'dim_ruido': [noise_dim],
    'feature_dim': [feature_dim],
    'timesteps': [time_steps],
    'epochs': [epochs],
    'batch_size': [batch_size],
    'model': [modelo],
    'alpha': [leaky_relu_alpha],
    'negative_slope': [leaky_relu_negative_slope],
    'dropuot': [dropout],
    'optimizador': [opt],
    'learning_rate':[lr],
    'loss': [loss],
    'weighted_thr': [threshold],
    'weighted_factor': [factor],
    'jensen_shannon': [jsd_value],
    'wasserstein_distance': [wasserstein_dist],
    'kolmogorov_smirnov': [ks_stat],
    'p_value': [p_value]
}

conf_res= pd.DataFrame(conf_res_data)

file_conf_res = f'config_results_{exp}_{prueba}.csv'

conf_res.to_csv(file_conf_res, index=False)

In [ ]:
file_turb_data= f'turb_data_{exp}_{prueba}.csv'
data_full[['date','turb_filled']].to_csv(file_turb_data, index=False)